__Analysis Script__

In [36]:
import pandas as pd
import os
import numpy as np

In [37]:
path="/home/maximilian/Master/carla_garage/logs"

In [38]:
summary="bcso_wp_old_weights.csv"

In [39]:
df=pd.read_csv(os.path.join(path, summary))

In [40]:
df

,town,traffic,weather,start,target,route_completion,lights_ran,duration,timeout,collision
0,Town01,1,1,19,103,28.80,0,39.60,0,1
1,Town01,1,1,78,225,28.47,0,83.90,0,1
2,Town01,1,1,103,21,46.91,0,88.75,0,1
3,Town01,1,1,127,87,30.48,0,45.35,0,1
4,Town01,1,1,230,210,100.00,0,5.45,0,0
5,Town01,1,1,250,190,43.68,0,65.90,0,1
6,Town01,1,1,220,118,85.18,0,441.05,1,0
7,Town01,1,1,200,224,8.81,0,95.55,0,1
8,Town01,1,1,11,17,11.38,0,38.85,0,1
9,Town01,1,1,78,245,24.20,0,129.10,0,1


In [41]:
threshold=.5

In [42]:
# (df["timeout"]-df["final_time"]<threshold).sum()
timeouts=(df["timeout"]==1).sum()

In [43]:
timeouts

2

In [44]:
#collisions=((df["end_other_collision"]) | (df["end_pedestrian_collision"]) | (df["end_vehicle_collision"])).sum()
collisions=(df["collision"]==1).sum()

In [45]:
collisions

19

In [46]:
success=len(df)-timeouts-collisions

In [47]:
success

4

In [10]:
from tqdm import tqdm
import regex as re
import os
import pandas as pd

In [4]:
print("Started scanning...")
result_files = {}
for root, dirs, files in os.walk("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input"):
    for dir in dirs:
        eval_reps = os.path.join(root, dir)
        if re.findall(".*_.*", dir):
            for rep_root, rep_dirs, rep_files in os.walk(eval_reps):
                for rep_dir in rep_dirs:
                    file_lst=[]
                    rep_path = os.path.join(eval_reps, rep_dir)
                    if rep_dir == "results":
                        for filename in tqdm(os.listdir(rep_path)):
                            file_lst.append(os.path.join(rep_path, filename))
                        result_files[dir] = file_lst

Started scanning...


100%|██████████| 832/832 [00:00<00:00, 856148.41it/s]


In [8]:
weathers = {"train": [1, 3, 6, 8], "test": [10, 14]}
def weather_mapping(value):
    for key, value_lst in weathers.items():
        if value in value_lst:
            return key

In [7]:
result_files

{'nocrash_e1_30': ['/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_w-train_t-Town02_r-1_t-repetition_0_s-coil_o-nocrash_Town02_split_5.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_w-train_t-Town02_r-1_t-repetition_2_s-all_o-nocrash_Town02_split_10.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_w-train_t-Town02_r-1_t-repetition_2_s-all_o-nocrash_Town02_split_13.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-bcoh_e-bcoh_w-train_t-Town01_r-1_t-repetition_2_s-all_o-nocrash_Town01_split_9.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-arp_e-arp_w-train_t-Town01_r-1_t-repetition_0_s-all_o-nocrash_Town01_split_10.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/results/nocrash_b-bcso_e-bcso_w-train_t-Town02_r-1_t-repetition_1_s-all_o-nocrash_Town02_split_5.csv',
  '/home/maximilian/Master/eval_test/nocrash_e1_30/result

In [15]:
df_lst=[]
for eval_rep, path_lst in tqdm(result_files.items()):
    for path in path_lst:
        eval_results = pd.read_csv(path)
        eval_results["eval_rep"] = eval_rep
        df_lst.append(eval_results)
df = pd.concat(df_lst, ignore_index=True)
df["success"] = (df["timeout"] == 0) & (df["collision"] == 0)
df["success"] = df["success"].astype("int")
df["weather"] = df["weather"].map(lambda x: weather_mapping(x))
groups = df.groupby(["baseline", "experiment", "town","traffic", "weather", "eval_rep", "setting"]).agg(
    timeouts_percentage=("timeout", "mean"),
    collisions_percentage=("collision", "mean"),
    success_percentage=("success", "mean"),
)
groups = groups.groupby(["baseline", "experiment", "town", "traffic", "weather", "setting"]).agg(
    timeout_mean=("timeouts_percentage", "mean"),
    timeout_std=("timeouts_percentage", "std"),
    collisions_mean=("collisions_percentage", "mean"),
    collisions_std=("collisions_percentage", "std"),
    success_mean=("success_percentage", "mean"),
    success_std=("success_percentage", "std"),
)

100%|██████████| 3/3 [00:03<00:00,  1.17s/it]


In [13]:
pd.set_option('display.max_rows', 500)

In [16]:
groups

timeout_mean  \
baseline experiment              town   traffic weather setting                     
arp      arp                     Town01 0       test    all              0.000000   
                                                        coil             0.013333   
                                                train   all              0.000000   
                                                        coil             0.040000   
                                        1       test    all              0.000000   
                                                        coil             0.046667   
                                                train   all              0.000000   
                                                        coil             0.041667   
                                        2       test    all              0.020000   
                                                        coil             0.050000   
                                                train   all              0.010000   
                                                        coil             0.043333   
                                 Town02 0       test    all              0.086667   
                                                        coil             0.163333   
                                                train   all              0.050000   
                                                        coil             0.206667   
                                        1       test    all              0.060000   
                                                        coil             0.166667   
                                                train   all              0.046667   
                                                        coil             0.206667   
                                        2       test    all              0.400000   
                                                        coil             0.283333   
                                                train   all              0.400000   
                                                        coil             0.278333   
         arp_nospeed_data_rep_1  Town01 0       test    02_withheld      0.000000   
                                                train   02_withheld      0.000000   
                                        1       test    02_withheld      0.033333   
                                                train   02_withheld      0.033333   
                                        2       test    02_withheld      0.060000   
                                                train   02_withheld      0.043333   
                                 Town02 0       test    02_withheld      0.033333   
                                                train   02_withheld      0.016667   
                                        1       test    02_withheld      0.040000   
                                                train   02_withheld      0.020000   
                                        2       test    02_withheld      0.280000   
                                                train   02_withheld      0.206667   
bcoh     bcoh                    Town01 0       test    all              0.000000   
                                                        coil             0.020000   
                                                train   all              0.000000   
                                                        coil             0.013333   
                                        1       test    all              0.010000   
                                                        coil             0.006667   
                                                train   all              0.005000   
                                                        coil             0.013333   
                                        2       test    all              0.000000   
                                                        coil             0

In [ ]:
groups.to_csv(os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input", os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input","combined.csv")))